In [2]:
import os, sys, pimms, pandas, warnings
from pathlib import Path

import numpy as np
import scipy as sp
import nibabel as nib
import neuropythy as ny

import matplotlib as mpl
import matplotlib.pyplot as plt
import ipyvolume as ipv
import torch
import skimage as ski
import sklearn as skl
from os import fspath

In [5]:
def filter_image(image, gabor_sizes=None, gabor_angles=None):
    """Filters the image with a set of Gabor filters and returns the filtered
    images as an array.
    """
    if gabor_sizes is None:
        gabor_sizes=[4, 6, 8, 12, 16, 24]
    if gabor_angles is None:
        gabor_angles = np.arange(8) / 8 * np.pi
    # Code
    im = image
    # Filter in many ways:
    output = []
    for gsz in gabor_sizes:
        sz_output = []
        for gang in gabor_angles:
            (real,imag) = ski.filters.gabor(im, theta=gang, frequency=1/gsz)
            sz_output.append(np.sqrt(real**2 + imag**2))
        sz_output = np.array(sz_output)
        #sz_output /= np.mean(sz_output, axis=0) + 0.5
        output.append(sz_output)
    output = np.array(output)
    return(output)
    

In [8]:
im = (ski.io.imread("/Users/ameliakeyser-gibson/Documents/SEFS/Kim Lab/vines grant/thermal/june 2025 images/RGB cropped/FLIR2211.jpg"))

In [13]:
flir_path = Path('/Users/ameliakeyser-gibson/Documents/SEFS/Kim Lab/vines grant/thermal/june 2025 images/RGB cropped/')
norm_sigma = 10
image_filenames = [
    p
    for p in flir_path.iterdir()
    if p.is_file()
    if p.name.endswith('.jpg')
    if not p.name.startswith('.')]
raw_images = np.array(
    [ski.io.imread(flnm)
     for flnm in image_filenames],
    dtype=float)
raw_images /= 255
bw_images = np.array([np.mean(im, axis=-1) for im in raw_images])

# Process them into green-accentuated images!
green_images = np.array([im[:,:,1] for im in raw_images], dtype=float)
redblue_conv_images = np.array(
    [ski.filters.gaussian((im[:,:,0] + im[:,:,2]) / 2, sigma=norm_sigma, preserve_range=True)
     for im in raw_images],
    dtype=float)
# Divisive normalized images:
images = green_images / (redblue_conv_images + 0.05)

In [15]:
filter_image(images[1])

array([[[[0.02748145, 0.03095777, 0.0347148 , ..., 0.00141213,
          0.00155356, 0.00179227],
         [0.02510911, 0.02825033, 0.0316686 , ..., 0.00136648,
          0.00149931, 0.00170631],
         [0.02074068, 0.02330147, 0.02620414, ..., 0.00131909,
          0.00139769, 0.00152209],
         ...,
         [0.00486966, 0.00344477, 0.00393597, ..., 0.00015616,
          0.00151079, 0.00252356],
         [0.00249289, 0.00331144, 0.00460427, ..., 0.0010153 ,
          0.00080843, 0.00168253],
         [0.00181335, 0.00436193, 0.00572206, ..., 0.00173432,
          0.00077261, 0.0011238 ]],

        [[0.01414784, 0.01735994, 0.02169264, ..., 0.00233315,
          0.0022942 , 0.00242921],
         [0.01325235, 0.01588981, 0.01926999, ..., 0.00194729,
          0.00211117, 0.00245796],
         [0.01157008, 0.01354025, 0.01574482, ..., 0.00153366,
          0.00203968, 0.00241892],
         ...,
         [0.00375255, 0.0029053 , 0.00412442, ..., 0.00350622,
          0.00412267, 0.0